In [ ]:
import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader

@st.cache_data
def load_data():
    books = pd.read_csv("updated_books.csv")
    ratings = pd.read_csv("synthetic_ratings.csv")
    books['combined'] = books['title'].fillna('') + ' ' + books['authors'].fillna('')
    return books, ratings

books, ratings = load_data()

# TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books['combined'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# SVD Model
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

def recommend_content(title, top_n=5):
    idx = books[books['title'].str.lower() == title.lower()].index
    if len(idx) == 0:
        return pd.DataFrame()
    idx = idx[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    book_indices = [i[0] for i in sim_scores]
    return books.iloc[book_indices][['title', 'authors']]

def recommend_collaborative(user_id, top_n=5):
    user_rated = ratings[ratings['user_id'] == user_id]['book_id'].tolist()
    unrated_books = books[~books['book_id'].isin(user_rated)]
    predictions = [algo.predict(user_id, bid) for bid in unrated_books['book_id']]
    predictions.sort(key=lambda x: x.est, reverse=True)
    top_books = [pred.iid for pred in predictions[:top_n]]
    return books[books['book_id'].isin(top_books)][['title', 'authors']]

def recommend_hybrid(user_id, title, top_n=5):
    content_recs = recommend_content(title, top_n=10)
    content_ids = books[books['title'].isin(content_recs['title'])]['book_id'].tolist()
    hybrid_preds = [algo.predict(user_id, bid) for bid in content_ids]
    hybrid_preds.sort(key=lambda x: x.est, reverse=True)
    top_ids = [pred.iid for pred in hybrid_preds[:top_n]]
    return books[books['book_id'].isin(top_ids)][['title', 'authors']]

st.title(" Book Recommendation Engine")

option = st.selectbox("Select Recommendation Type", ["Content-Based", "Collaborative", "Hybrid"])

if option == "Content-Based":
    title = st.text_input("Enter Book Title:")
    if title:
        recs = recommend_content(title)
        st.dataframe(recs)

elif option == "Collaborative":
    user_id = st.number_input("Enter User ID:", min_value=1, step=1)
    if user_id:
        recs = recommend_collaborative(user_id)
        st.dataframe(recs)

elif option == "Hybrid":
    title = st.text_input("Enter Book Title:")
    user_id = st.number_input("Enter User ID:", min_value=1, step=1)
    if title and user_id:
        recs = recommend_hybrid(user_id, title)
        st.dataframe(recs)
